Этот блокнот позволяет поиграться с моделькой, классифицирующей тексты как токсичные или неполиткорректные. 

Всё самое весёлое - в последней ячейке. 

https://huggingface.co/cointegrated/rubert-tiny-toxicity

In [ ]:
!pip install transformers sentencepiece --quiet

     |████████████████████████████████| 2.6 MB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
     |████████████████████████████████| 636 kB 43.2 MB/s 
     |████████████████████████████████| 895 kB 29.2 MB/s 
     |████████████████████████████████| 3.3 MB 42.3 MB/s 


In [1]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/home/dshome/anaconda3/envs/auto_gptq_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

In [3]:
def text2toxicity(text, aggregate=True):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1]) # степень токсичности.
    return proba

print(text2toxicity('я люблю нигеров', True))

print(text2toxicity('я люблю нигеров', False))

print(text2toxicity(['я люблю нигеров', 'я люблю африканцев'], True))

print(text2toxicity(['я люблю нигеров', 'я люблю африканцев'], False))

0.9350118648354808
[0.9715759  0.01808627 0.0045551  0.00189755 0.9331106 ]
[0.93501186 0.04156357]
[[9.7157592e-01 1.8086256e-02 4.5550931e-03 1.8975530e-03 9.3311059e-01]
 [9.9979788e-01 1.9048377e-04 1.5297388e-04 1.7452303e-04 4.1369792e-02]]


In [5]:
text2toxicity('я люблю цветы', True)

0.017520160843575994

In [15]:
ar = text2toxicity('я люблю нигеров', False)

In [9]:
name_value = ['non-toxic', 'insult', 'obscenity', 'threat', 'dangerous']

In [17]:
values  = list(filter(lambda x: x[0] > 0.9, zip(ar, name_value)))

[(0.9715759, 'non-toxic'), (0.9331106, 'dangerous')]

In [18]:
value = list(map(lambda y: y[1],
          list(filter(lambda x: x[0] > 0.9, zip(ar, name_value)))))

['non-toxic', 'dangerous']